In [1]:
# Data Manipulation and Preparation
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import csv
from random import randint, uniform, choice
import itertools

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Evaluation metrics and visualisation
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Ensure GPU usage
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

import time

Using device: cuda


In [2]:
# Load cleaned dataset
spot_df = pd.read_csv('eth_1h_cleaned.csv', parse_dates=['open_time'])

spot_simple_features = ['open', 'high', 'low', 'close', 'volume']

simple_spot_df = spot_df[spot_simple_features].copy()

In [3]:
pd.set_option('display.float_format', '{:.6f}'.format)

In [5]:
# Function to create sequences
def create_sequences(data, target, window_size):
    sequences = []
    labels = []
    for i in range(len(data) - window_size):
        seq = data[i:i + window_size]
        label = target[i + window_size]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

# LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_dim, lstm_dim, dense_dim, output_dim, num_layers=1, dropout=0.0):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, lstm_dim, num_layers=num_layers, batch_first=True, dropout=(dropout if num_layers > 1 else 0))
        self.fc1 = nn.Linear(lstm_dim, dense_dim)
        self.fc2 = nn.Linear(dense_dim, output_dim)
    
    def forward(self, x):
        x, _ = self.lstm(x)
        x = x[:, -1, :] 
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        return x

def train_and_evaluate(window_size=10, lstm_dim=50, dense_dim=50, num_layers=1, dropout=0.0, lr=0.001, batch_size=64, num_epochs=1000, optimizer_type='Adam', patience=10):
    # Clear CUDA cache
    torch.cuda.empty_cache()
    
    # Prepare data
    X = simple_spot_df.values
    y = simple_spot_df['close'].values.reshape(-1, 1)  # Reshape y to match scaler's expected input

    scaler_X = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X)

    scaler_y = MinMaxScaler()
    y_scaled = scaler_y.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, shuffle=False)
    X_train_seq, y_train_seq = create_sequences(X_train, y_train, window_size)
    X_test_seq, y_test_seq = create_sequences(X_test, y_test, window_size)

    X_train_tensor = torch.tensor(X_train_seq, dtype=torch.float32).to(device)
    X_test_tensor = torch.tensor(X_test_seq, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train_seq, dtype=torch.float32).view(-1, 1).to(device)
    y_test_tensor = torch.tensor(y_test_seq, dtype=torch.float32).view(-1, 1).to(device)

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

    # Model parameters
    input_dim = X_train_tensor.shape[2]
    output_dim = 1

    model = LSTMModel(input_dim, lstm_dim, dense_dim, output_dim, num_layers, dropout).to(device)

    # Define loss
    criterion = nn.MSELoss()
    
    # Select optimizer
    if optimizer_type == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=lr)
    elif optimizer_type == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=lr)
    elif optimizer_type == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=lr)
    else:
        raise ValueError("Unsupported optimizer type")

    # Training loop
    best_loss = float('inf')
    patience_counter = 0
    start_time = time.time()
    
    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                outputs = model(inputs)
                val_loss += criterion(outputs, labels).item()
        
        val_loss /= len(test_loader)
        
        if val_loss < best_loss:
            best_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                break

    training_time = time.time() - start_time

    model.load_state_dict(torch.load('best_model.pth'))

    # Evaluate the model
    model.eval()
    evaluation_start_time = time.time()
    with torch.no_grad():
        train_predictions = model(X_train_tensor).cpu().numpy()
        test_predictions = model(X_test_tensor).cpu().numpy()

        # Inverse transform predictions and actual values
        train_predictions_inverse = scaler_y.inverse_transform(train_predictions)
        test_predictions_inverse = scaler_y.inverse_transform(test_predictions)
        y_train_inverse = scaler_y.inverse_transform(y_train_seq)
        y_test_inverse = scaler_y.inverse_transform(y_test_seq)

        train_mse = mean_squared_error(y_train_inverse, train_predictions_inverse)
        test_mse = mean_squared_error(y_test_inverse, test_predictions_inverse)
        train_mae = mean_absolute_error(y_train_inverse, train_predictions_inverse)
        test_mae = mean_absolute_error(y_test_inverse, test_predictions_inverse)
        train_rmse = np.sqrt(train_mse)
        test_rmse = np.sqrt(test_mse)
        train_r2 = r2_score(y_train_inverse, train_predictions_inverse)
        test_r2 = r2_score(y_test_inverse, test_predictions_inverse)
        
        # Calculate accuracy-like metric with tighter threshold
        def calculate_accuracy(y_true, y_pred, percentage=0.01):
            range_y = y_true.max() - y_true.min()
            threshold = percentage * range_y
            diff = np.abs(y_true - y_pred)
            accuracy = np.mean(diff < threshold)
            return accuracy

        train_acc = calculate_accuracy(y_train_inverse, train_predictions_inverse)
        test_acc = calculate_accuracy(y_test_inverse, test_predictions_inverse)

    evaluation_time = time.time() - evaluation_start_time

    result = {
        "window_size": window_size,
        "lstm_dim": lstm_dim,
        "num_layers": num_layers,
        "dense_dim": dense_dim,
        "dropout": dropout,
        "lr": lr,
        "batch_size": batch_size,
        "num_epochs": num_epochs,
        "optimizer_type": optimizer_type,
        "train_mse": train_mse,
        "test_mse": test_mse,
        "train_mae": train_mae,
        "test_mae": test_mae,
        "train_rmse": train_rmse,
        "test_rmse": test_rmse,
        "train_r2": train_r2,
        "test_r2": test_r2,
        "train_acc": train_acc,
        "test_acc": test_acc,
        "training_time": training_time,
        "evaluation_time": evaluation_time,
        "patience": patience  # Add patience to the result
    }

    # Save result to a CSV file
    with open('results_ETH_param2.csv', 'a', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=result.keys())
        if f.tell() == 0:
            writer.writeheader()  # Write header only if file is empty
        writer.writerow(result)

    # Print summary of the results
    print(f"Finished model with parameters: {param_dict}")
    print(f"Results: Train MSE: {train_mse:.4f}, Test MSE: {test_mse:.4f}, Train MAE: {train_mae:.4f}, Test MAE: {test_mae:.4f}")
    print(f"Train RMSE: {train_rmse:.4f}, Test RMSE: {test_rmse:.4f}, Train R2: {train_r2:.4f}, Test R2: {test_r2:.4f}")
    print(f"Train Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}")
    print(f"Training Time: {training_time:.4f} seconds, Evaluation Time: {evaluation_time:.4f} seconds\n")

    return result

In [6]:
# Parameter space definition
parameter_space = {
    'window_size': [24, 36],
    'lstm_dim': [48, 64, 96],
    'dense_dim': [96, 120],
    'num_layers': [1],
    'dropout': [0.005, 0.01, 0.02],
    'lr': [0.001, 0.002],
    'batch_size': [64],
    'optimizer_type': ['Adam'],
    'patience': [17, 20, 23]
}

# Grid search
for params in itertools.product(*parameter_space.values()):
    param_dict = dict(zip(parameter_space.keys(), params))
    print(f"Running model with parameters: {param_dict}")
    train_and_evaluate(**param_dict)

Running model with parameters: {'window_size': 24, 'lstm_dim': 48, 'dense_dim': 96, 'num_layers': 1, 'dropout': 0.005, 'lr': 0.001, 'batch_size': 64, 'optimizer_type': 'Adam', 'patience': 17}
Finished model with parameters: {'window_size': 24, 'lstm_dim': 48, 'dense_dim': 96, 'num_layers': 1, 'dropout': 0.005, 'lr': 0.001, 'batch_size': 64, 'optimizer_type': 'Adam', 'patience': 17}
Results: Train MSE: 398.2741, Test MSE: 236.0528, Train MAE: 11.3053, Test MAE: 8.7807
Train RMSE: 19.9568, Test RMSE: 15.3640, Train R2: 0.9997, Test R2: 0.9995
Train Accuracy: 0.9628, Test Accuracy: 0.9273
Training Time: 50.4593 seconds, Evaluation Time: 0.0751 seconds

Running model with parameters: {'window_size': 24, 'lstm_dim': 48, 'dense_dim': 96, 'num_layers': 1, 'dropout': 0.005, 'lr': 0.001, 'batch_size': 64, 'optimizer_type': 'Adam', 'patience': 20}
Finished model with parameters: {'window_size': 24, 'lstm_dim': 48, 'dense_dim': 96, 'num_layers': 1, 'dropout': 0.005, 'lr': 0.001, 'batch_size': 64,

In [18]:
# Function to create sequences
def create_sequences(data, target, window_size):
    sequences = []
    labels = []
    for i in range(len(data) - window_size):
        seq = data[i:i + window_size]
        label = target[i + window_size]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

class LSTMModel(nn.Module):
    def __init__(self, input_dim, lstm_dim, dense_dim, output_dim, num_layers=1, dropout=0.0, activation_function=nn.ReLU):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, lstm_dim, num_layers=num_layers, batch_first=True, dropout=(dropout if num_layers > 1 else 0))
        self.fc1 = nn.Linear(lstm_dim, dense_dim)
        self.activation = activation_function()
        self.fc2 = nn.Linear(dense_dim, output_dim)
    
    def forward(self, x):
        x, _ = self.lstm(x)
        x = x[:, -1, :] 
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        return x


def train_and_evaluate(window_size=10, lstm_dim=50, dense_dim=50, num_layers=1, dropout=0.0, lr=0.001, batch_size=64, num_epochs=1000, optimizer_type='Adam', patience=10, activation_function=nn.ReLU):
    # Clear CUDA cache
    torch.cuda.empty_cache()
    
    # Prepare data
    X = simple_spot_df.values
    y = simple_spot_df['close'].values.reshape(-1, 1)  # Reshape y to match scaler's expected input

    scaler_X = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X)

    scaler_y = MinMaxScaler()
    y_scaled = scaler_y.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, shuffle=False)
    X_train_seq, y_train_seq = create_sequences(X_train, y_train, window_size)
    X_test_seq, y_test_seq = create_sequences(X_test, y_test, window_size)

    X_train_tensor = torch.tensor(X_train_seq, dtype=torch.float32).to(device)
    X_test_tensor = torch.tensor(X_test_seq, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train_seq, dtype=torch.float32).view(-1, 1).to(device)
    y_test_tensor = torch.tensor(y_test_seq, dtype=torch.float32).view(-1, 1).to(device)

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

    # Model parameters
    input_dim = X_train_tensor.shape[2]
    output_dim = 1

    model = LSTMModel(input_dim, lstm_dim, dense_dim, output_dim, num_layers, dropout, activation_function).to(device)

    # Define loss
    criterion = nn.MSELoss()
    
    # Select optimizer
    if optimizer_type == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=lr)
    elif optimizer_type == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=lr)
    elif optimizer_type == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=lr)
    elif optimizer_type == 'SGD with Momentum':
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    elif optimizer_type == 'AdamW':
        optimizer = optim.AdamW(model.parameters(), lr=lr)
    else:
        raise ValueError("Unsupported optimizer type")

    # Training loop
    best_loss = float('inf')
    patience_counter = 0
    start_time = time.time()
    
    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                outputs = model(inputs)
                val_loss += criterion(outputs, labels).item()
        
        val_loss /= len(test_loader)
        
        if val_loss < best_loss:
            best_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                break

    training_time = time.time() - start_time

    model.load_state_dict(torch.load('best_model.pth'))

    # Evaluate the model
    model.eval()
    evaluation_start_time = time.time()
    with torch.no_grad():
        train_predictions = model(X_train_tensor).cpu().numpy()
        test_predictions = model(X_test_tensor).cpu().numpy()

        # Inverse transform predictions and actual values
        train_predictions_inverse = scaler_y.inverse_transform(train_predictions)
        test_predictions_inverse = scaler_y.inverse_transform(test_predictions)
        y_train_inverse = scaler_y.inverse_transform(y_train_seq)
        y_test_inverse = scaler_y.inverse_transform(y_test_seq)

        train_mse = mean_squared_error(y_train_inverse, train_predictions_inverse)
        test_mse = mean_squared_error(y_test_inverse, test_predictions_inverse)
        train_mae = mean_absolute_error(y_train_inverse, train_predictions_inverse)
        test_mae = mean_absolute_error(y_test_inverse, test_predictions_inverse)
        train_rmse = np.sqrt(train_mse)
        test_rmse = np.sqrt(test_mse)
        train_r2 = r2_score(y_train_inverse, train_predictions_inverse)
        test_r2 = r2_score(y_test_inverse, test_predictions_inverse)
        
        # Calculate accuracy-like metric with tighter threshold
        def calculate_accuracy(y_true, y_pred, percentage=0.01):
            range_y = y_true.max() - y_true.min()
            threshold = percentage * range_y
            diff = np.abs(y_true - y_pred)
            accuracy = np.mean(diff < threshold)
            return accuracy

        train_acc = calculate_accuracy(y_train_inverse, train_predictions_inverse)
        test_acc = calculate_accuracy(y_test_inverse, test_predictions_inverse)

    evaluation_time = time.time() - evaluation_start_time

    result = {
        "window_size": window_size,
        "lstm_dim": lstm_dim,
        "num_layers": num_layers,
        "dense_dim": dense_dim,
        "dropout": dropout,
        "lr": lr,
        "batch_size": batch_size,
        "num_epochs": num_epochs,
        "optimizer_type": optimizer_type,
        "activation_function": activation_function.__name__,
        "train_mse": train_mse,
        "test_mse": test_mse,
        "train_mae": train_mae,
        "test_mae": test_mae,
        "train_rmse": train_rmse,
        "test_rmse": test_rmse,
        "train_r2": train_r2,
        "test_r2": test_r2,
        "train_acc": train_acc,
        "test_acc": test_acc,
        "training_time": training_time,
        "evaluation_time": evaluation_time,
        "patience": patience  # Add patience to the result
    }

    # Save result to a CSV file
    with open('results_ETH_param3.csv', 'a', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=result.keys())
        if f.tell() == 0:
            writer.writeheader()  # Write header only if file is empty
        writer.writerow(result)

    # Print summary of the results
    print(f"Finished model with parameters: {param_dict}")
    print(f"Results: Train MSE: {train_mse:.4f}, Test MSE: {test_mse:.4f}, Train MAE: {train_mae:.4f}, Test MAE: {test_mae:.4f}")
    print(f"Train RMSE: {train_rmse:.4f}, Test RMSE: {test_rmse:.4f}, Train R2: {train_r2:.4f}, Test R2: {test_r2:.4f}")
    print(f"Train Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}")
    print(f"Training Time: {training_time:.4f} seconds, Evaluation Time: {evaluation_time:.4f} seconds\n")

    return result

In [19]:
# Add activation functions to the parameter space
parameter_space = {
    'window_size': [24],
    'lstm_dim': [64, 96, 128],
    'dense_dim': [96],
    'num_layers': [1, 2],
    'dropout': [0.0, 0.01],
    'lr': [0.001, 0.0005, 0.0001],
    'batch_size': [64],
    'optimizer_type': ['Adam', 'AdamW'],
    'patience': [25],
    'activation_function': [nn.ReLU, nn.LeakyReLU, nn.Tanh]
}

In [20]:
import winsound

def play_beep():
    duration = 1000
    freq = 440 
    winsound.Beep(freq, duration)

In [21]:
# Grid search
for params in itertools.product(*parameter_space.values()):
    param_dict = dict(zip(parameter_space.keys(), params))
    print(f"Running model with parameters: {param_dict}")
    train_and_evaluate(**param_dict)

play_beep()

Running model with parameters: {'window_size': 24, 'lstm_dim': 64, 'dense_dim': 96, 'num_layers': 1, 'dropout': 0.0, 'lr': 0.001, 'batch_size': 64, 'optimizer_type': 'Adam', 'patience': 25, 'activation_function': <class 'torch.nn.modules.activation.ReLU'>}
Finished model with parameters: {'window_size': 24, 'lstm_dim': 64, 'dense_dim': 96, 'num_layers': 1, 'dropout': 0.0, 'lr': 0.001, 'batch_size': 64, 'optimizer_type': 'Adam', 'patience': 25, 'activation_function': <class 'torch.nn.modules.activation.ReLU'>}
Results: Train MSE: 419.9398, Test MSE: 235.2473, Train MAE: 13.2084, Test MAE: 8.8490
Train RMSE: 20.4924, Test RMSE: 15.3378, Train R2: 0.9997, Test R2: 0.9995
Train Accuracy: 0.9637, Test Accuracy: 0.9278
Training Time: 57.0810 seconds, Evaluation Time: 0.0996 seconds

Running model with parameters: {'window_size': 24, 'lstm_dim': 64, 'dense_dim': 96, 'num_layers': 1, 'dropout': 0.0, 'lr': 0.001, 'batch_size': 64, 'optimizer_type': 'Adam', 'patience': 25, 'activation_function':